In [35]:
import json
import pandas as pd
from mm_analytics.utilities import NpEncoder
DATA_ROOT = "/Users/andrewgrowney/data/kaggle/marchmadness-2023/Stage2"

In [15]:
seasons_df = pd.read_csv(f"{DATA_ROOT}/MSeasons.csv") # Season, DayZero, RegionW, RegionX, RegionY, RegionZ
seeds_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneySeeds.csv") # Season, Seed, TeamID
slots_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneySlots.csv") # Season, Slot, StrongSeed, WeakSeed
seed_round_slots_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneySeedRoundSlots.csv") # Seed, GameRound, GameSlot, EarlyDayNum, LateDayNum
results_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneyCompactResults.csv") # Season, DayNum, WTeamID, LTeamID, WScore, LScore, WLoc, NumOT

In [27]:
# Slots
# R1W1 vs R1W16
# R1W8 vs R1W9    # R2W1 vs R2W8
# R1W5 vs R1W12   # R2W1 vs R2W4
# R1W4 vs R1W13
# R1W6 vs R1W11
# R1W3 vs R1W14   # R2W3 vs R3W4
# R1W2 vs R1W15
# ...
slots_df[(slots_df["Season"] == 2022)].head(5)


,Season,Slot,StrongSeed,WeakSeed
2318,2022,R1W1,W01,W16
2319,2022,R1W2,W02,W15
2320,2022,R1W3,W03,W14
2321,2022,R1W4,W04,W13
2322,2022,R1W5,W05,W12


In [28]:
tournament["slots"].keys()

KeyError: 'slots'

In [53]:
# Build out json view of the tournament from the dataframes
for season in range(2013, 2018):
    print(f"Season: {season}")
    seasion_regions = seasons_df[seasons_df["Season"] == season]
    season_slots = slots_df[slots_df["Season"] == season]
    season_seeds = seeds_df[seeds_df["Season"] == season]
    season_results = results_df[results_df["Season"] == season]
    season_teams = season_seeds["TeamID"].unique()
    season_teams.sort()
    print(f"Teams: {len(season_teams)}")
    print(f"Slots: {len(season_slots)}")
    print(f"SeedRoundSlots: {len(seed_round_slots_df)}")
    print(f"Seeds: {len(season_seeds)}")
    print(f"Regions: {seasion_regions}")

    # Build out json view of the tournament from the dataframes
    # Use slots to build out the tournament structure
    # Each slot has a strong seed and a weak seed
    tournament = {
        "regions": {
            "W": seasion_regions["RegionW"].values[0],
            "X": seasion_regions["RegionX"].values[0],
            "Y": seasion_regions["RegionY"].values[0],
            "Z": seasion_regions["RegionZ"].values[0]
        },
        "slots": {}
    }
    for index, row in season_slots.iterrows():
        slot = row["Slot"]
        strong_seed = row["StrongSeed"]
        weak_seed = row["WeakSeed"]
        seed_round_slots = seed_round_slots_df[seed_round_slots_df["GameSlot"] == slot]
        if slot not in tournament["slots"]:
            tournament["slots"][slot] = {}
        tournament["slots"][slot]["round_start"] = seed_round_slots["EarlyDayNum"].values[0]
        tournament["slots"][slot]["strong_seed"] = strong_seed
        tournament["slots"][slot]["weak_seed"] = weak_seed
    
    # Fill Team Seeds Mapping
    seed_teams = {}
    for index, row in season_seeds.iterrows():
        seed = row["Seed"]
        team_id = row["TeamID"]
        seed_teams[seed] = team_id

    # Fill in the first round slot
    # strong seed and week seed data from seeds data
    slot_winners = {}
    slot_keys_sorted_by_day_range = sorted(tournament["slots"].keys(), key=lambda x: tournament["slots"][x]["round_start"])
    for slot in slot_keys_sorted_by_day_range:
        try:
            strong_seed = tournament["slots"][slot]["strong_seed"]
            if strong_seed.startswith("R") or strong_seed in slot_winners:
                # Fetch game winner
                strong_seed_team_id = slot_winners[strong_seed]
            else:
                # Fetch from seeding data
                strong_seed_team_id = seed_teams[strong_seed]

            weak_seed = tournament["slots"][slot]["weak_seed"]
            if weak_seed.startswith("R") or weak_seed in slot_winners:
                weak_seed_team_id = slot_winners[weak_seed]
            else:
                weak_seed_team_id = seed_teams[weak_seed]

            print(f"{strong_seed}[{strong_seed_team_id}] vs {weak_seed}[{weak_seed_team_id}][{season}]")
            tournament["slots"][slot]["strong_seed"] = strong_seed_team_id
            tournament["slots"][slot]["weak_seed"] = weak_seed_team_id
            # Fetch slot winner from seaon_results
            if season == 2021 and slot == "R1X7":
                # VCU vs Oregon forfeit
                slot_winner, wscore, lscore = 1332, 2, 0
            else:
                game_row = season_results[season_results["WTeamID"].isin([strong_seed_team_id, weak_seed_team_id]) & season_results["LTeamID"].isin([strong_seed_team_id, weak_seed_team_id])]
                slot_winner = game_row["WTeamID"].values[0]
                wscore, lscore = game_row["WScore"].values[0], game_row["LScore"].values[0]
            tournament["slots"][slot]["winner"] = slot_winner
            tournament["slots"][slot]["wscore"] = wscore
            tournament["slots"][slot]["lscore"] = lscore

            print(f"{slot} winner: {slot_winner} [{game_row['WScore'].values[0]}-{game_row['LScore'].values[0]}]")
            slot_winners[slot] = slot_winner
        except Exception as e:
            print(f"Error {e}: {slot} {strong_seed} {weak_seed}, {slot_winners}")
            raise

    with open(f"../data/web/tourney/{season}.json", "w") as f:
        json.dump(tournament, f, indent=4, cls=NpEncoder)

Season: 2018
Teams: 68
Slots: 67
SeedRoundSlots: 720
Seeds: 68
Regions:     Season     DayZero RegionW  RegionX RegionY RegionZ
33    2018  2017-10-30    East  Midwest   South    West
W11a[1382] vs W11b[1417][2018]
W11 winner: 1382 [65-58]
W16a[1254] vs W16b[1347][2018]
W16 winner: 1347 [71-61]
X11a[1113] vs X11b[1393][2018]
X11 winner: 1393 [60-56]
Z16a[1300] vs Z16b[1411][2018]
Z16 winner: 1411 [64-46]
W01[1437] vs W16[1347][2018]
R1W1 winner: 1437 [87-61]
W02[1345] vs W15[1168][2018]
R1W2 winner: 1345 [74-48]
W03[1403] vs W14[1372][2018]
R1W3 winner: 1403 [70-60]
W04[1455] vs W13[1267][2018]
R1W4 winner: 1267 [81-75]
W05[1452] vs W12[1293][2018]
R1W5 winner: 1452 [85-68]
W06[1196] vs W11[1382][2018]
R1W6 winner: 1196 [77-62]
W07[1116] vs W10[1139][2018]
R1W7 winner: 1139 [79-62]
W08[1439] vs W09[1104][2018]
R1W8 winner: 1104 [86-83]
X01[1242] vs X16[1335][2018]
R1X1 winner: 1242 [76-60]
X02[1181] vs X15[1233][2018]
R1X2 winner: 1181 [89-67]
X03[1277] vs X14[1137][2018]
R1X3 winner: 